In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import activation, Dropout, Dense
from keras.layers import Flatten, LSTM, CuDNNLSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional
import re

In [2]:
#Import the new dataset modified to 365 days
x_train = pd.read_csv('Power365.csv')
y_train = pd.read_csv('AlwaysOn365.csv')
x_test = pd.read_csv('x_test30.csv')
y_test = pd.read_csv('y_test30.csv')

In [3]:
x_train

,power
0,73270
1,73330
2,73210
3,73130
4,72980
...,...
15767995,288870
15767996,270830
15767997,235440
15767998,227970


In [4]:
y_train

,applianceType,startTime,stopTime,usage,unit
0,AlwaysOn,1509494400,1509580800,4.1516,kWh
1,AlwaysOn,1509580800,1509667200,4.0285,kWh
2,AlwaysOn,1509667200,1509753600,4.8795,kWh
3,AlwaysOn,1509753600,1509840000,4.2236,kWh
4,AlwaysOn,1509840000,1509926400,3.9257,kWh
...,...,...,...,...,...
360,AlwaysOn,1540425600,1540512000,4.7498,kWh
361,AlwaysOn,1540512000,1540598400,9.2016,kWh
362,AlwaysOn,1540598400,1540684800,5.7921,kWh
363,AlwaysOn,1540684800,1540771200,4.5130,kWh


In [5]:
#scaler = MinMaxScaler()
x = np.array(x_train.power)
#x = x.reshape(-1, 1)
X_train = x.reshape(365,43200,1) #converts the list X into 3-dimensional shape with 500 samples, 43200 time-steps, and 1 feature
X_train = X_train.astype("float32")
X_test = np.array(x_test.power)
X_test = X_test.reshape(30,43200)
X_test = X_test.astype("float32")

Y_train = np.array(y_train.usage)
Y_train = Y_train.astype("float32")
Y_test = np.array(y_test.usage)
Y_test = Y_test.astype("float32")
#Y = Y.array.reshape(-1, 1)


In [6]:
print(Y_train.ndim)
print(Y_test.ndim)
print(X_train.ndim)
print(X_test.ndim)

1
1
3
2


In [7]:
print("Num GPU Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

Num GPU Available:  1
True


In [8]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [23]:
#Create the model LSTM Bidirectional
model = Sequential()
model.add(LSTM(10, input_shape=(43200, 1), activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10)                480       
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 491
Trainable params: 491
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(optimizer='adam', loss='mse')

In [25]:
history = model.fit(X_train, Y_train, epochs=10, validation_split=0.4, verbose=1, batch_size=1)

Epoch 1/10
  8/219 [>.............................] - ETA: 3:51:53 - loss: 8545080320.0000

KeyboardInterrupt: 

In [ ]:
model.save('modelo3.h5')

In [ ]:
# Para cargar el modelo
#from keras.models import load_model
#model = load_model('modelo1.h5')

In [14]:
evaluation_result = model.evaluate(X_test, Y_test, batch_size=1)


30/30 [==============================] - 4s 108ms/step - loss: 1.5641


In [21]:
X_test = X_test.reshape(30,43200,1)
test_output = model.predict(X_test, verbose=1, batch_size=1)
print(test_output)
print(test_output.shape)

30/30 [==============================] - 3s 113ms/step
[[3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]
 [3.275568]]
(30, 1)


In [15]:
x_axis = np.arange(0, len(test_output))
plt.plot(x_axis, test_output) 
plt.plot(x_axis, Y_test)

plt.title("Comparación entre predicción y Ground truth")
plt.xlabel("Día")
plt.ylabel("Consumo [kWh]") 

plt.show()

NameError: name 'test_output' is not defined